In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
from google.colab import files
import io
import math

# Use this to upload your CSV file directly to the Colab environment
uploaded = files.upload()
file_name = next(iter(uploaded)) # Get the name of the uploaded file

# Load the data into a Pandas DataFrame
df = pd.read_csv(io.StringIO(uploaded[file_name].decode('utf-8')))

# Rename columns to match the required schema for consistency
# 'type' in CSV maps to 'class' in schema. 'timestamp' maps to 't'.
df = df.rename(columns={'type': 'class', 'timestamp': 't'})

# Convert 't' to datetime objects and set as index (good for time-series analysis)
df['t'] = pd.to_datetime(df['t'])
# df = df.set_index('t').sort_index()

print("✅ Data Loaded Successfully.")
df.info()
print("\nFirst 5 rows:")
print(df.head())

Saving work-package-raw-data.csv to work-package-raw-data (1).csv
✅ Data Loaded Successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   class        100000 non-null  object             
 1   x            100000 non-null  float64            
 2   y            100000 non-null  float64            
 3   id           100000 non-null  int64              
 4   speed        100000 non-null  float64            
 5   heading      100000 non-null  float64            
 6   area         100000 non-null  int64              
 7   vest         100000 non-null  int64              
 8   with_object  100000 non-null  bool               
 9   t            100000 non-null  datetime64[ns, UTC]
dtypes: bool(1), datetime64[ns, UTC](1), float64(4), int64(3), object(1)
memory usage: 7.0+ MB

First 5 rows:
     class        x       

In [ ]:
# Confirm unique values in key columns
print("\nUnique Classes (for filtering/grouping):")
print(df['class'].unique())

print("\nUnique Areas/Zones (for filtering/grouping):")
print(df['area'].unique())

print("\nVest Status Counts (0=No Vest, 1=Vest Worn):")
print(df['vest'].value_counts(dropna=False))

print(f"\nTotal Unique Assets (id): {df['id'].nunique()}")


Unique Classes (for filtering/grouping):
['vehicle' 'human']

Unique Areas/Zones (for filtering/grouping):
[ 2  1 12  9  8 11 13  4  5  3  7]

Vest Status Counts (0=No Vest, 1=Vest Worn):
vest
0    81989
1    18011
Name: count, dtype: int64

Total Unique Assets (id): 106


In [ ]:
# Filter for Vest Violations
vest_violations_df = df[(df['class'] == 'human') & (df['vest'] == 0)]

total_vest_violations = len(vest_violations_df)
total_human_detections = len(df[df['class'] == 'human'])

print(f"Total Human Detections: {total_human_detections}")
print(f"Total Vest Violations (vest=0): {total_vest_violations}")

# SQL Equivalent (using SQLite syntax placeholder):
# SELECT COUNT(*) FROM detections WHERE class = 'human' AND vest = 0

Total Human Detections: 28323
Total Vest Violations (vest=0): 10312


In [ ]:
# Define threshold
OVERSPEED_THRESHOLD = 1.5

# Filter for Overspeed Events
overspeed_df = df[df['speed'] > OVERSPEED_THRESHOLD]

total_overspeed_events = len(overspeed_df)

print(f"Total Overspeed Events (speed > {OVERSPEED_THRESHOLD} m/s): {total_overspeed_events}")
print(f"Overspeed Events by Class:\n{overspeed_df['class'].value_counts()}")

# SQL Equivalent:
# SELECT COUNT(*), class FROM detections WHERE speed > 1.5 GROUP BY class

Total Overspeed Events (speed > 1.5 m/s): 16599
Overspeed Events by Class:
class
human      15013
vehicle     1586
Name: count, dtype: int64


## Filter and count

### Subtask:
Filter the merged pairs to keep only those with a distance less than or equal to 2.0m and count the resulting pairs to get the total number of close calls.

**Reasoning**:
Filter the merged dataframe to find close calls based on the distance threshold and then print the total number of these close calls.

In [ ]:
# Filter for close calls based on the distance threshold
close_calls_df = merged_df[merged_df['distance'] <= 2.0]

# Calculate the total number of close calls
total_close_calls = len(close_calls_df)

# Print the total number of close calls
print(f"Total Close Calls (distance <= 2.0m): {total_close_calls}")

Total Close Calls (distance <= 2.0m): 0


## Distance calculation

### Subtask:
Calculate the Euclidean distance between the 'x' and 'y' coordinates for each merged pair.

**Reasoning**:
Calculate the Euclidean distance between the human and vehicle coordinates in the merged dataframe.

In [ ]:
# Calculate the difference in x and y coordinates
delta_x = merged_df['x_human'] - merged_df['x_vehicle']
delta_y = merged_df['y_human'] - merged_df['y_vehicle']

# Calculate the Euclidean distance
merged_df['distance'] = np.sqrt(delta_x**2 + delta_y**2)

print("Merged DataFrame with calculated distances:")
display(merged_df.head())

Merged DataFrame with calculated distances:


,class_human,x_human,y_human,id_human,speed_human,heading_human,area_human,vest_human,with_object_human,t,class_vehicle,x_vehicle,y_vehicle,id_vehicle,speed_vehicle,heading_vehicle,area_vehicle,vest_vehicle,with_object_vehicle,distance
0,human,7.4930,17.5302,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,130.191338
1,human,2.1956,17.9105,122378,1.3880,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,130.107034
2,human,1.6554,23.0477,122411,1.7911,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,125.029320
3,human,14.3674,18.2606,122423,5.1473,0.0,9,0,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,129.399267
4,human,7.4907,17.5276,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.503000+00:00,vehicle,13.0257,147.4176,15,0.0,6.5816,1,0,False,130.007878


## Temporal join

### Subtask:
Merge the human and vehicle DataFrames based on the 't' column with a time tolerance of 250ms.

**Reasoning**:
Merge the human and vehicle dataframes based on the 't' column with a time tolerance of 250ms to find potential close encounters.

In [ ]:
# Perform temporal merge with a 250ms tolerance
merged_df = pd.merge_asof(
    human_df.sort_values('t'),
    vehicle_df.sort_values('t'),
    on='t',
    tolerance=pd.Timedelta('250ms'),
    direction='nearest',
    suffixes=('_human', '_vehicle')
)

print("Merged DataFrame with potential close calls:")
display(merged_df.head())
print(f"\nNumber of merged detections: {len(merged_df)}")

Merged DataFrame with potential close calls:


,class_human,x_human,y_human,id_human,speed_human,heading_human,area_human,vest_human,with_object_human,t,class_vehicle,x_vehicle,y_vehicle,id_vehicle,speed_vehicle,heading_vehicle,area_vehicle,vest_vehicle,with_object_vehicle
0,human,7.4930,17.5302,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
1,human,2.1956,17.9105,122378,1.3880,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
2,human,1.6554,23.0477,122411,1.7911,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
3,human,14.3674,18.2606,122423,5.1473,0.0,9,0,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
4,human,7.4907,17.5276,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.503000+00:00,vehicle,13.0257,147.4176,15,0.0,6.5816,1,0,False



Number of merged detections: 28323


## Isolate classes

### Subtask:
Create separate DataFrames for 'human' and 'vehicle' class types.

**Reasoning**:
The subtask requires filtering the main DataFrame into two separate DataFrames based on the 'class' column. This can be achieved by using boolean indexing in pandas.

In [ ]:
human_df = df[df['class'] == 'human'].copy()
vehicle_df = df[df['class'] == 'vehicle'].copy()

print(f"Number of human detections: {len(human_df)}")
print(f"Number of vehicle detections: {len(vehicle_df)}")

Number of human detections: 28323
Number of vehicle detections: 71677


## Summary:

### Data Analysis Key Findings

* The initial data contained 28323 human detections and 71677 vehicle detections.
* A temporal merge was performed between human and vehicle detections that occurred within 250ms of each other.
* The Euclidean distance was calculated for each merged human-vehicle pair.
* After filtering for pairs with a distance less than or equal to 2.0m, the total number of identified close calls was 0.
* The total number of Vest Violations (human detections with vest=0) was 10312.
* The total number of Overspeed Events (detections with speed > 1.5 m/s) was 16599, with the breakdown by class being:
    * Human: 15013
    * Vehicle: 1586


### Insights or Next Steps

* Investigate if a wider time tolerance or distance threshold for close calls is more appropriate for identifying potential interactions, given the current analysis yielded 0 close calls.
* Explore alternative methods for associating human and vehicle detections, such as spatial proximity in addition to temporal proximity, especially if increasing the thresholds still results in a low number of close calls.
* Consider the implications of the data collection methodology and sensor accuracy on the close call analysis.
* Use the confirmed logic from this analysis to implement the KPI calculations efficiently in the backend using optimized SQL queries.

In [ ]:
print(total_vest_violations)
print({total_close_calls})
print(overspeed_df['class'].value_counts().get('human', 0))
print(overspeed_df['class'].value_counts().get('vehicle', 0))
print({total_overspeed_events})
print({OVERSPEED_THRESHOLD})
{len(vehicle_df)}
{total_human_detections}

10312
{0}
15013
1586
{16599}
{1.5}


{28323}

# Task
Modify the empty cell with id "ef6c53da" to include Python code that calculates the number of close calls based on the provided conceptual steps. The code should isolate human and vehicle data, perform a temporal merge with a 250ms tolerance, calculate the distance between human and vehicle coordinates, filter for distances less than or equal to 2.0m, and print the total count of close calls.

## Isolate classes

### Subtask:
Create separate DataFrames for 'human' and 'vehicle' class types.


**Reasoning**:
The subtask requires filtering the main DataFrame into two separate DataFrames based on the 'class' column. This can be achieved by using boolean indexing in pandas.



In [ ]:
human_df = df[df['class'] == 'human'].copy()
vehicle_df = df[df['class'] == 'vehicle'].copy()

print(f"Number of human detections: {len(human_df)}")
print(f"Number of vehicle detections: {len(vehicle_df)}")

Number of human detections: 28323
Number of vehicle detections: 71677


## Temporal join

### Subtask:
Merge the human and vehicle DataFrames based on the 't' column with a time tolerance of 250ms.


**Reasoning**:
Merge the human and vehicle dataframes based on the 't' column with a time tolerance of 250ms to find potential close encounters.



In [ ]:
# Perform temporal merge with a 250ms tolerance
merged_df = pd.merge_asof(
    human_df.sort_values('t'),
    vehicle_df.sort_values('t'),
    on='t',
    tolerance=pd.Timedelta('250ms'),
    direction='nearest',
    suffixes=('_human', '_vehicle')
)

print("Merged DataFrame with potential close calls:")
display(merged_df.head())
print(f"\nNumber of merged detections: {len(merged_df)}")

Merged DataFrame with potential close calls:


,class_human,x_human,y_human,id_human,speed_human,heading_human,area_human,vest_human,with_object_human,t,class_vehicle,x_vehicle,y_vehicle,id_vehicle,speed_vehicle,heading_vehicle,area_vehicle,vest_vehicle,with_object_vehicle
0,human,7.4930,17.5302,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
1,human,2.1956,17.9105,122378,1.3880,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
2,human,1.6554,23.0477,122411,1.7911,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
3,human,14.3674,18.2606,122423,5.1473,0.0,9,0,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
4,human,7.4907,17.5276,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.503000+00:00,vehicle,13.0257,147.4176,15,0.0,6.5816,1,0,False



Number of merged detections: 28323


## Distance calculation

### Subtask:
Calculate the Euclidean distance between the 'x' and 'y' coordinates for each merged pair.


**Reasoning**:
Calculate the Euclidean distance between the human and vehicle coordinates in the merged dataframe.



In [ ]:
# Calculate the difference in x and y coordinates
delta_x = merged_df['x_human'] - merged_df['x_vehicle']
delta_y = merged_df['y_human'] - merged_df['y_vehicle']

# Calculate the Euclidean distance
merged_df['distance'] = np.sqrt(delta_x**2 + delta_y**2)

print("Merged DataFrame with calculated distances:")
display(merged_df.head())

Merged DataFrame with calculated distances:


,class_human,x_human,y_human,id_human,speed_human,heading_human,area_human,vest_human,with_object_human,t,class_vehicle,x_vehicle,y_vehicle,id_vehicle,speed_vehicle,heading_vehicle,area_vehicle,vest_vehicle,with_object_vehicle,distance
0,human,7.4930,17.5302,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,130.191338
1,human,2.1956,17.9105,122378,1.3880,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,130.107034
2,human,1.6554,23.0477,122411,1.7911,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,125.029320
3,human,14.3674,18.2606,122423,5.1473,0.0,9,0,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,129.399267
4,human,7.4907,17.5276,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.503000+00:00,vehicle,13.0257,147.4176,15,0.0,6.5816,1,0,False,130.007878


## Filter and count

### Subtask:
Filter the merged pairs to keep only those with a distance less than or equal to 2.0m and count the resulting pairs to get the total number of close calls.


**Reasoning**:
Filter the merged dataframe to find close calls based on the distance threshold and then print the total number of these close calls.



In [ ]:
# Filter for close calls based on the distance threshold
close_calls_df = merged_df[merged_df['distance'] <= 2.0]

# Calculate the total number of close calls
total_close_calls = len(close_calls_df)

# Print the total number of close calls
print(f"Total Close Calls (distance <= 2.0m): {total_close_calls}")

Total Close Calls (distance <= 2.0m): 0


## Summary:

### Data Analysis Key Findings

*   The initial data contained 28,323 human detections and 71,677 vehicle detections.
*   A temporal merge was performed between human and vehicle detections that occurred within 250ms of each other.
*   The Euclidean distance was calculated for each merged human-vehicle pair.
*   After filtering for pairs with a distance less than or equal to 2.0m, the total number of identified close calls was 0.

### Insights or Next Steps

*   Investigate if a wider time tolerance or distance threshold is more appropriate for identifying potential interactions.
*   Explore alternative methods for associating human and vehicle detections, such as spatial proximity in addition to temporal proximity.


In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
# from google.colab import files # Not needed when reading directly from path
import io
import math

# Use this to upload your CSV file directly to the Colab environment
# uploaded = files.upload() # Removed interactive file upload
file_name = '/content/work-package-raw-data.csv' # Use the direct path

# Load the data into a Pandas DataFrame
# df = pd.read_csv(io.StringIO(uploaded[file_name].decode('utf-8'))) # Modified to read directly from path
df = pd.read_csv(file_name)


# Rename columns to match the required schema for consistency
# 'type' in CSV maps to 'class' in schema. 'timestamp' maps to 't'.
df = df.rename(columns={'type': 'class', 'timestamp': 't'})

# Convert 't' to datetime objects and set as index (good for time-series analysis)
df['t'] = pd.to_datetime(df['t'])
# df = df.set_index('t').sort_index()

print("✅ Data Loaded Successfully.")
df.info()
print("\nFirst 5 rows:")
print(df.head())

✅ Data Loaded Successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   class        100000 non-null  object             
 1   x            100000 non-null  float64            
 2   y            100000 non-null  float64            
 3   id           100000 non-null  int64              
 4   speed        100000 non-null  float64            
 5   heading      100000 non-null  float64            
 6   area         100000 non-null  int64              
 7   vest         100000 non-null  int64              
 8   with_object  100000 non-null  bool               
 9   t            100000 non-null  datetime64[ns, UTC]
dtypes: bool(1), datetime64[ns, UTC](1), float64(4), int64(3), object(1)
memory usage: 7.0+ MB

First 5 rows:
     class        x         y      id  speed   heading  area  vest  \
0  vehicle  22.3764  1

In [ ]:


# Confirm unique values in key columns
print("\nUnique Classes (for filtering/grouping):")
print(df['class'].unique())

print("\nUnique Areas/Zones (for filtering/grouping):")
print(df['area'].unique())

print("\nVest Status Counts (0=No Vest, 1=Vest Worn):")
print(df['vest'].value_counts(dropna=False))

print(f"\nTotal Unique Assets (id): {df['id'].nunique()}")


# Filter for Vest Violations
vest_violations_df = df[(df['class'] == 'human') & (df['vest'] == 0)]

total_vest_violations = len(vest_violations_df)
total_human_detections = len(df[df['class'] == 'human'])

print(f"\nTotal Human Detections: {total_human_detections}")
print(f"Total Vest Violations (vest=0): {total_vest_violations}")

# SQL Equivalent (using SQLite syntax placeholder):
# SELECT COUNT(*) FROM detections WHERE class = 'human' AND vest = 0


# Define threshold
OVERSPEED_THRESHOLD = 1.5

# Filter for Overspeed Events
overspeed_df = df[df['speed'] > OVERSPEED_THRESHOLD]

total_overspeed_events = len(overspeed_df)

print(f"\nTotal Overspeed Events (speed > {OVERSPEED_THRESHOLD} m/s): {total_overspeed_events}")
print(f"Overspeed Events by Class:\n{overspeed_df['class'].value_counts()}")

# SQL Equivalent:
# SELECT COUNT(*), class FROM detections WHERE speed > 1.5 GROUP BY class


# Close Calls Calculation Steps

# 1. Isolate Classes
human_df = df[df['class'] == 'human'].copy()
vehicle_df = df[df['class'] == 'vehicle'].copy()

print(f"\nNumber of human detections: {len(human_df)}")
print(f"Number of vehicle detections: {len(vehicle_df)}")

# 2. Temporal Join
# Perform temporal merge with a 250ms tolerance
merged_df = pd.merge_asof(
    human_df.sort_values('t'),
    vehicle_df.sort_values('t'),
    on='t',
    tolerance=pd.Timedelta('250ms'),
    direction='nearest',
    suffixes=('_human', '_vehicle')
)

print("\nMerged DataFrame with potential close calls:")
display(merged_df.head())
print(f"\nNumber of merged detections: {len(merged_df)}")

# 3. Distance Calculation
# Calculate the difference in x and y coordinates
delta_x = merged_df['x_human'] - merged_df['x_vehicle']
delta_y = merged_df['y_human'] - merged_df['y_vehicle']

# Calculate the Euclidean distance
merged_df['distance'] = np.sqrt(delta_x**2 + delta_y**2)

print("\nMerged DataFrame with calculated distances:")
display(merged_df.head())

# 4. Filter and count
# Filter for close calls based on the distance threshold
close_calls_df = merged_df[merged_df['distance'] <= 2.0]

# Calculate the total number of close calls
total_close_calls = len(close_calls_df)

print(f"\nTotal Close Calls (distance <= 2.0m): {total_close_calls}")


Unique Classes (for filtering/grouping):
['vehicle' 'human']

Unique Areas/Zones (for filtering/grouping):
[ 2  1 12  9  8 11 13  4  5  3  7]

Vest Status Counts (0=No Vest, 1=Vest Worn):
vest
0    81989
1    18011
Name: count, dtype: int64

Total Unique Assets (id): 106

Total Human Detections: 28323
Total Vest Violations (vest=0): 10312

Total Overspeed Events (speed > 1.5 m/s): 16599
Overspeed Events by Class:
class
human      15013
vehicle     1586
Name: count, dtype: int64

Number of human detections: 28323
Number of vehicle detections: 71677

Merged DataFrame with potential close calls:


,class_human,x_human,y_human,id_human,speed_human,heading_human,area_human,vest_human,with_object_human,t,class_vehicle,x_vehicle,y_vehicle,id_vehicle,speed_vehicle,heading_vehicle,area_vehicle,vest_vehicle,with_object_vehicle
0,human,7.4930,17.5302,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
1,human,2.1956,17.9105,122378,1.3880,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
2,human,1.6554,23.0477,122411,1.7911,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
3,human,14.3674,18.2606,122423,5.1473,0.0,9,0,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False
4,human,7.4907,17.5276,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.503000+00:00,vehicle,13.0257,147.4176,15,0.0,6.5816,1,0,False



Number of merged detections: 28323

Merged DataFrame with calculated distances:


,class_human,x_human,y_human,id_human,speed_human,heading_human,area_human,vest_human,with_object_human,t,class_vehicle,x_vehicle,y_vehicle,id_vehicle,speed_vehicle,heading_vehicle,area_vehicle,vest_vehicle,with_object_vehicle,distance
0,human,7.4930,17.5302,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,130.191338
1,human,2.1956,17.9105,122378,1.3880,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,130.107034
2,human,1.6554,23.0477,122411,1.7911,0.0,12,1,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,125.029320
3,human,14.3674,18.2606,122423,5.1473,0.0,9,0,False,2025-04-02 15:42:06.435000+00:00,vehicle,12.1000,147.6400,122261,0.0,0.0000,1,0,False,129.399267
4,human,7.4907,17.5276,122380,0.0000,0.0,12,1,False,2025-04-02 15:42:06.503000+00:00,vehicle,13.0257,147.4176,15,0.0,6.5816,1,0,False,130.007878



Total Close Calls (distance <= 2.0m): 0


## Summary:

### Data Analysis Key Findings

* The initial data contained 28323 human detections and 71677 vehicle detections.
* A temporal merge was performed between human and vehicle detections that occurred within 250ms of each other.
* The Euclidean distance was calculated for each merged human-vehicle pair.
* After filtering for pairs with a distance less than or equal to 2.0m, the total number of identified close calls was 0.
* The total number of Vest Violations (human detections with vest=0) was 10312.
* The total number of Overspeed Events (detections with speed > 1.5 m/s) was 16599, with the breakdown by class being:
    * Human: 15013
    * Vehicle: 1586


### Insights or Next Steps

* Investigate if a wider time tolerance or distance threshold for close calls is more appropriate for identifying potential interactions, given the current analysis yielded 0 close calls.
* Explore alternative methods for associating human and vehicle detections, such as spatial proximity in addition to temporal proximity, especially if increasing the thresholds still results in a low number of close calls.
* Consider the implications of the data collection methodology and sensor accuracy on the close call analysis.
* Use the confirmed logic from this analysis to implement the KPI calculations efficiently in the backend using optimized SQL queries.

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
# from google.colab import files # Not needed when reading directly from path
import io
import math

# Use this to upload your CSV file directly to the Colab environment
# uploaded = files.upload() # Removed interactive file upload
file_name = '/content/work-package-raw-data.csv' # Use the direct path

# Load the data into a Pandas DataFrame
# df = pd.read_csv(io.StringIO(uploaded[file_name].decode('utf-8'))) # Modified to read directly from path
df = pd.read_csv(file_name)


# Rename columns to match the required schema for consistency
# 'type' in CSV maps to 'class' in schema. 'timestamp' maps to 't'.
df = df.rename(columns={'type': 'class', 'timestamp': 't'})

# Ensure 't' is datetime and 'vest' and 'area' are appropriate types for filtering
df['t'] = pd.to_datetime(df['t'])
df['vest'] = df['vest'].astype('Int64') # Use Int64 for nullable integer (0 or 1)
df['area'] = df['area'].astype(str)

print("✅ Data Loaded and Schema Aligned.")
df.info()

✅ Data Loaded and Schema Aligned.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 10 columns):
 #   Column       Non-Null Count   Dtype              
---  ------       --------------   -----              
 0   class        100000 non-null  object             
 1   x            100000 non-null  float64            
 2   y            100000 non-null  float64            
 3   id           100000 non-null  int64              
 4   speed        100000 non-null  float64            
 5   heading      100000 non-null  float64            
 6   area         100000 non-null  object             
 7   vest         100000 non-null  Int64              
 8   with_object  100000 non-null  bool               
 9   t            100000 non-null  datetime64[ns, UTC]
dtypes: Int64(1), bool(1), datetime64[ns, UTC](1), float64(4), int64(1), object(2)
memory usage: 7.1+ MB


In [ ]:
print("\n--- Scenario 1: Vest Violations Grouped by Hour ---")

# 1. Apply Filters (SQL WHERE clause equivalent)
filtered_df = df[(df['class'] == 'human') & (df['vest'] == 0)].copy()

# 2. Apply Grouping (SQL GROUP BY clause equivalent: date_trunc('hour', t))
# Use pd.Grouper to bucket time by frequency ('H' for hour)
hourly_violations = filtered_df.groupby(
    pd.Grouper(key='t', freq='H') # Grouper acts like date_trunc
)['id'].count().reset_index() # Count the records in each bucket

hourly_violations.columns = ['time_bucket', 'violations_count']

print(hourly_violations.head())

# Verification: The sum should match your ground truth (10312)
print(f"Total violations across all hours: {hourly_violations['violations_count'].sum()}")


--- Scenario 1: Vest Violations Grouped by Hour ---
                time_bucket  violations_count
0 2025-04-02 15:00:00+00:00             10295
1 2025-04-02 16:00:00+00:00                17
Total violations across all hours: 10312


/tmp/ipython-input-182439720.py:9: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  pd.Grouper(key='t', freq='H') # Grouper acts like date_trunc


In [ ]:
print("\n--- Scenario 2: Vest Violations Grouped by Area ---")

# 1. Apply Filters (Same as Scenario 1's WHERE clause)
filtered_df = df[(df['class'] == 'human') & (df['vest'] == 0)]

# 2. Apply Grouping (SQL GROUP BY area equivalent)
area_violations = filtered_df.groupby('area')['id'].count().reset_index()
area_violations.columns = ['area', 'violations_count']

# Sort to find the "Top Risky" areas
top_risky_areas = area_violations.sort_values(by='violations_count', ascending=False)

print("Top 5 Risky Areas by Vest Violations:")
print(top_risky_areas.head())

# Verification: The sum should match your ground truth (10312)
print(f"Total violations across all areas: {top_risky_areas['violations_count'].sum()}")


--- Scenario 2: Vest Violations Grouped by Area ---
Top 5 Risky Areas by Vest Violations:
  area  violations_count
0   11              5824
1   12              3892
3    2               144
4    3               140
9    9               134
Total violations across all areas: 10312


In [ ]:
print("\n--- Scenario 6: Close Call Trend (Complex Join Logic) ---")
# Close Call Parameters
DISTANCE_THRESHOLD = 2.0  # meters
TIME_TOLERANCE_MS = 250   # milliseconds (0.250 seconds)

# 1. Isolate Classes
humans = df[df['class'] == 'human'].rename(columns={'x': 'hx', 'y': 'hy', 't': 'ht', 'id': 'human_id'})
vehicles = df[df['class'].isin(['vehicle', 'pallet_truck', 'agv'])].rename(columns={'x': 'vx', 'y': 'vy', 't': 'vt', 'id': 'vehicle_id'})

# 2. Temporal Join (Requires specialized tools or an iterative approach in Pandas)
# Since a proper time-interval join is highly complex in basic Pandas, we use a simplified merge
# and then manually apply the time tolerance, mirroring the required SQL logic:
# JOIN vehicles v ON v.t BETWEEN h.t - 250ms AND h.t + 250ms

# Set a time key for rough merging (e.g., 5-second bucket) to reduce join size
humans['time_key'] = humans['ht'].dt.floor('5s')
vehicles['time_key'] = vehicles['vt'].dt.floor('5s')

# Merge on the rough time key
merged = pd.merge(humans, vehicles, on='time_key', how='inner')

# 3. Apply Fine-Grained Temporal Filter (Temporal WHERE clause)
# Calculate time difference
merged['time_diff_ms'] = (merged['ht'] - merged['vt']).abs().dt.total_seconds() * 1000
temporal_filtered = merged[merged['time_diff_ms'] <= TIME_TOLERANCE_MS].copy() # Explicitly create a copy

# 4. Calculate Distance and Apply Spatial Filter (Spatial WHERE clause)
# Euclidean distance: sqrt((hx - vx)^2 + (hy - vy)^2)
temporal_filtered['distance'] = np.sqrt(
    (temporal_filtered['hx'] - temporal_filtered['vx'])**2 +
    (temporal_filtered['hy'] - temporal_filtered['vy'])**2
)
close_calls = temporal_filtered[temporal_filtered['distance'] <= DISTANCE_THRESHOLD]

# 5. Apply Grouping (SQL GROUP BY date_trunc('day', t))
daily_close_calls = close_calls.groupby(
    pd.Grouper(key='ht', freq='D') # Group by human time 'ht'
)['human_id'].count().reset_index()

daily_close_calls.columns = ['day_bucket', 'close_call_count']

print(f"Total Close Calls Identified (Distance <= {DISTANCE_THRESHOLD}m, Time <= {TIME_TOLERANCE_MS}ms): {daily_close_calls['close_call_count'].sum()}")
print("Daily Close Call Counts:")
print(daily_close_calls)

# Verification: The sum must match your ground truth (0).


--- Scenario 6: Close Call Trend (Complex Join Logic) ---
Total Close Calls Identified (Distance <= 2.0m, Time <= 250ms): 0
Daily Close Call Counts:
Empty DataFrame
Columns: [day_bucket, close_call_count]
Index: []


## Summary of Vest Violations and Close Call Analyses

### Key Findings:

*   **Vest Violations**: The total number of Vest Violations (human detections with vest=0) identified in the dataset is {total_vest_violations}.
*   **Close Calls**: After performing a temporal merge (within 250ms) of human and vehicle detections and filtering for spatial proximity (distance <= 2.0m), the total number of identified close calls is {total_close_calls}.

### Insights:

*   The analysis shows a significant number of vest violations among human detections.
*   The close call analysis, using the specified temporal and spatial thresholds, resulted in 0 identified close calls. This suggests that either the thresholds might be too strict for this dataset or the data does not contain instances meeting these specific close call criteria. Further investigation with wider thresholds or alternative proximity definitions might be necessary to capture potential interactions.